 # Tutorial of the molecule builder for ESPResSo: pyMBE





The objective of pyMBE is to facilitate the building of complex molecules into the Molecular Dynamics software ESPResSo. Furthermore, it includes several handy tools to facilitate the setting up of molecular dynamics and constant pH simulations in ESPResSo. In other words: adding sugar to your ESPResSo scripts makes your life sweater. 

## Table of contents:
* [Introduction](#introduction)
* [Use pyMBE as a molecule builder for ESPResSo](#molecule_builder)
    * [Create particles](#particles)
    * [Create polymers](#polymers)
    * [Create peptides](#peptides)

## Introduction <a class="anchor" id="introduction"></a>

Let us get started by importing pyMBE library and creating an instance of it. 

In [1]:
import pyMBE
pmb = pyMBE.pymbe_library()

When sugar is inicialitated, a default system of reduced units is defined. The active set of reduced units can be consulted using

In [2]:
pmb.print_reduced_units()


Current set of reduced units:
0.355 nanometer = 1 reduced_length
4.11640356238e-21 joule = 1 reduced_energy
Temperature: 298.15 kelvin
1.602176634e-19 coulomb = 1 reduced_charge



This default definition of reduced units can be changed at the convience of the user using the command

In [3]:
pmb.set_reduced_units(unit_length=0.5*pmb.units.nm,  unit_charge=2*pmb.units.e, temperature=300 * pmb.units.K)


Current set of reduced units:
0.49999999999999994 nanometer = 1 reduced_length
4.141945559999999e-21 joule = 1 reduced_energy
Temperature: 300 kelvin
3.204353268e-19 coulomb = 1 reduced_charge



All input variables will be given to ESPResSo using these reduced units, since it is a convinient choice for the simulation setup. Internally, pyMBE uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables

In [4]:
Box_L = 7.5 * pmb.units.nm

print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

import espressomd

# Create an instance of an ESPResSo system 

espresso_system=espressomd.System(box_l=[Box_L.to('reduced_length').magnitude]*3)

The side of the simulation box is  7.5 nanometer = 14.999999999999998 reduced_length


## Use pyMBE as a molecule builder for ESPResSo <a class="anchor" id="molecule_builder"></a>
### Create particles <a class="anchor" id="particles"></a>

Particles are the smaller objects in the simulation box, which can represent small ions or other small chemical species. In turn, particles can also be used as building blocks for larger molecules or polymers, where they can represent one monomeric unit or part of it. Particle objects are used in pyMBE as input for several of its funcionalities, including to create larger molecules and peptides. The basic properties of a particle are:

In [5]:
cation_name = 'Na'
pmb.define_particle(name=cation_name, 
            q=1, 
            diameter=0.35*pmb.units.nm, 
            epsilon=1*pmb.units('reduced_energy'))

#Dataframe display with all the information of the particles

print (pmb.filter_df(pmb_type ='particle'))

  name  pmb_type acidity        diameter           epsilon state_one          \
                                                               label es_type   
0   Na  particle   inert  0.35 nanometer  1 reduced_energy        Na       0   

          
  charge  
0      1  


One can use pyMBE to create any number of the defined particles into the espresso system

In [6]:
N_cations=20
pmb.create_pmb_object_in_espresso(name=cation_name,
                                  number_of_objects=N_cations,
                                  espresso_system=espresso_system)

Let us see the particles that we have created by visualizing our espresso system

In [7]:
def do_snapshot_espresso_system(espresso_system, filename):
      """
      Uses espresso visualizator for creating a snapshot of the current state of the espresso_system
      """ 
      from espressomd import visualization
      

      visualizer = visualization.openGLLive(
            espresso_system, bond_type_radius=[0.3], particle_coloring='type', draw_axis=False, background_color=[1, 1, 1],
      particle_type_colors=[[1.02,0.51,0], # Brown
                        [1,1,1],  # Grey
                        [2.55,0,0], # Red
                        [0,0,2.05],  # Blue
                        [0,0,2.05],  # Blue
                        [2.55,0,0], # Red
                        [2.05,1.02,0]]) # Orange
      visualizer.screenshot(filename)

      return

In [8]:
picture_name='cation_system.png'
do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=cation_name, espresso_system=espresso_system)


### Use pyMBE to create polymers in ESPResSo <a class="anchor" id="polymers"></a>

pyMBE can be used to easily construct coarse-grained models of polymers with complex structures. Let us consider a coarse grained model for polyhydroalanaline which represents a monomer with three beads, as depicted in the schematics below: a backbone bead (purple), a bead for the carboxylic acid group (cyan) and a bead for the amino group (blue).|
<img src="figs/PDha.png" width=150 height=150 />


To set up such polymer with pyMBE first one has to define the different particles in the monomer

In [9]:
# Define each different bead in the polymer

PDha_backbone_bead = 'BB'
PDha_carboxyl_bead = 'COOH-PDha'
PDha_amine_bead = 'NH3-PDha'

pmb.define_particle(name=PDha_backbone_bead, q=0, 
                   diameter=0.4*pmb.units.nm, 
                   epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle(name=PDha_carboxyl_bead, q=0, 
                   diameter=0.5*pmb.units.nm, 
                   epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle(name=PDha_amine_bead, q=0, 
                   diameter=0.3*pmb.units.nm, 
                   epsilon=1*pmb.units('reduced_energy'))


Then, one defines the structure of the residue of the polymer. A residue is composed by a `central_bead` where one or various `side_chains` are attached. Each side chain can contain one particle or other residues. 

In [10]:
# Define the monomer residue

PDha_residue = 'PDha_mon'
pmb.define_residue(name = PDha_residue, central_bead = PDha_backbone_bead ,
                    side_chains=[PDha_carboxyl_bead,PDha_amine_bead])


Once done, one has to define a bond for each different bond in the polymer. For simplicity, in this tutorial we assume that all bonds are equal and we set-up all bonds using the same generic bond

In [11]:
# Define the bonds between the particles

from espressomd import interactions

generic_bond_lenght=0.5 * pmb.units.nm
generic_harmonic_constant=400*pmb.units('reduced_energy / nm**2') 
generic_bond = interactions.HarmonicBond(k=generic_harmonic_constant.to('reduced_energy / reduced_length**2').magnitude,
                                 r_0=generic_bond_lenght.to('reduced_length').magnitude)

# backbone bond
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_backbone_bead)
# backbone-COOH bond
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_carboxyl_bead)
# backbone-NH4 bond
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_amine_bead)

pmb.add_bonds_to_espresso(espresso_system=espresso_system)

Finally, one uses the residues to define the polymer sequence given by the argument `residue_list`. Note that one needs to add one residue in `residue_list` per each residue in your polymer chain, i.e.

In [12]:
# Define the polymer molecule

N_monomers=8

PDha_polymer = 'PDha'
pmb.define_molecule(name= PDha_polymer,residue_list=[PDha_residue]*N_monomers)

# Create a PdHa polymer into the espresso system

N_pol=1

pmb.create_pmb_object_in_espresso (name=PDha_polymer, 
                                   number_of_objects=N_pol,
                                   espresso_system=espresso_system, 
                                   position=[[Box_L.to('reduced_length').magnitude/2]*3]) #Create it in the box center 

picture_name='PDha_system.png'

do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=PDha_polymer, espresso_system=espresso_system)

pyMBE can also be used also to setup models that requiere more complex side chains, i.e. with more than one bead per side chain. One example of use of such a model is poly(N,N-diallylglutamate) (PDAGA), whose structure is depicted in the figure below. Following the logic of the previous example, one would construct PDAGA with pyMBE by defining a `residue` with a `central_bead` for the polymer backbone (grey) and a side chain attached to it. In this case, however, the group in the side chain of the PDAGA monomer has a complex structure. This group can be coarse-grained using pyMBE by defining it as a `residue`  whose `central_bead` is the cyclic amine group (orange) with two carboxyl beads attached on its `side_chains` (purple and yellow).
<img src="figs/PDAGA.png" width=150 height=150 />

In [13]:

# Define the different beads to model PDAGA 
# If not es_type is provided, pyMBE will asign one for you

PDAGA_backbone_bead='BB-PDAGA'

PDAGA_amine_bead = 'NH3-PDAGA'
PDAGA_alpha_carboxyl_bead ='aCOOH-PDAGA'
PDAGA_beta_carboxyl_bead ='bCOOH-PDAGA'

pmb.define_particle(name=PDAGA_backbone_bead, 
                    q=0,diameter=0.4*pmb.units.nm, 
                    epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_amine_bead, 
                   q=0, 
                   diameter=0.3*pmb.units.nm, 
                   epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_alpha_carboxyl_bead, 
                   q=0, 
                   diameter=0.2*pmb.units.nm, 
                   epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle(name= PDAGA_beta_carboxyl_bead, 
                   q = 0, 
                   diameter = 0.4*pmb.units.nm, 
                   epsilon = 1*pmb.units('reduced_energy'))

# Define a residue for the side chain of PDAGA
PDAGA_side_chain_residue = 'PDAGA_side_chain_residue'
pmb.define_residue (name=PDAGA_side_chain_residue,
                    central_bead=PDAGA_amine_bead,
                    side_chains=[PDAGA_alpha_carboxyl_bead,PDAGA_beta_carboxyl_bead])

# Define a residue for the PDAGA monomer
PDAGA_monomer_residue = 'PDAGA_monomer_residue'
pmb.define_residue( name = PDAGA_monomer_residue,
                    central_bead = PDAGA_backbone_bead,
                    side_chains = [PDAGA_side_chain_residue])

# Define the bonds between the particles

# backbone-backbone bond
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_backbone_bead)
# backbone-cyclic amine bond
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_amine_bead)
# cyclic amine - alpha carboxyl bond
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_alpha_carboxyl_bead, particle_name2=PDAGA_amine_bead)
# cyclic amine - beta carboxyl bond
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_beta_carboxyl_bead, particle_name2=PDAGA_amine_bead)

pmb.add_bonds_to_espresso(espresso_system=espresso_system)

# Define the polymer molecule

N_monomers=8
PDAGA_polymer = 'PDAGA'
pmb.define_molecule(name=PDAGA_polymer,residue_list=[PDAGA_monomer_residue]*N_monomers)

# Create a PdHa polymer into the espresso system

N_pol=1

pmb.create_pmb_object_in_espresso(name = PDAGA_polymer,
                                  number_of_objects=N_pol,
                                  espresso_system=espresso_system,
                                  position=[[Box_L.to('reduced_length').magnitude/2]*3]) #Create it in the box center 

picture_name='PDAGA_system.png'
do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=PDAGA_polymer, espresso_system=espresso_system)


In turn, the residues previously defined to build the PDAGA and PDha molecules can be used to build more complex polymers such as a di-block PDha-PDAGA copolymer, as shown in the picture below
<img src="figs/PDAGA_PDha_diblock_copolymer.png" width=250 height=250 />

In [14]:
# Define the bond between the backnone of PDha and PDAGA

PDha_backbone_bead = 'BB'
PDha_carboxyl_bead = 'COOH-PDha'
PDha_amine_bead = 'NH3-PDha'

PDAGA_backbone_bead='BB-PDAGA'
PDAGA_amine_bead = 'NH3-PDAGA'
PDAGA_alpha_carboxyl_bead ='aCOOH-PDAGA'
PDAGA_beta_carboxyl_bead ='bCOOH-PDAGA'

# Pdha backbone- PDAGA backbone bond

pmb.define_particle(name = PDha_backbone_bead, q=0,diameter=0.4*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle(name = PDha_carboxyl_bead, q=0, diameter=0.5*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle(name = PDha_amine_bead, q=0,diameter=0.3*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle( name = PDAGA_backbone_bead, q=0,diameter=0.4*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_amine_bead,   q = 0, diameter=0.3*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_alpha_carboxyl_bead, q = 0, diameter=0.2*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_beta_carboxyl_bead, q = 0, diameter = 0.4*pmb.units.nm, epsilon = 1*pmb.units('reduced_energy'))

# Define a residue for the side chain of PDAGA

PDAGA_side_chain_residue = 'PDAGA_side_chain_residue'
pmb.define_residue (name=PDAGA_side_chain_residue,
                    central_bead=PDAGA_amine_bead,
                    side_chains=[PDAGA_alpha_carboxyl_bead,PDAGA_beta_carboxyl_bead])

pmb.define_residue(name = PDha_residue, central_bead =  PDha_backbone_bead ,
                    side_chains=[PDha_carboxyl_bead,PDha_amine_bead])

pmb.define_residue( name = PDAGA_monomer_residue,
                    central_bead = PDAGA_backbone_bead,
                    side_chains = [PDAGA_side_chain_residue])

pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDAGA_backbone_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDAGA_backbone_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_carboxyl_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_amine_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_backbone_bead)

pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_backbone_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_amine_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_alpha_carboxyl_bead, particle_name2=PDAGA_amine_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_beta_carboxyl_bead, particle_name2=PDAGA_amine_bead)

pmb.add_bonds_to_espresso(espresso_system=espresso_system)

# Define the diblock polymer molecule

N_monomers_PDha=4
N_monomers_PDAGA=4
diblock_polymer='diblock'

pmb.define_molecule(name = diblock_polymer,residue_list=[PDha_residue]*N_monomers_PDha+[PDAGA_monomer_residue]*N_monomers_PDAGA)

# Create the diblock polymer into the espresso system

N_pol=1

pmb.create_pmb_object_in_espresso(name = diblock_polymer,
                                  number_of_objects=N_pol,
                                  espresso_system=espresso_system,
                                  position=[[Box_L.to('reduced_length').magnitude/2]*3]) #Create it in the box center 

picture_name='diblock_system.png'
do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=diblock_polymer, espresso_system=espresso_system)


Similarly, one can set-up an alternating PDha-PDAGA co-polymer, which has the structure depicted below

<img src="figs/PDAGA_PDha_alt_copolymer.png" width=250 height=250 />

In [15]:
# Define the alternating polymer molecule

N_monomers=4
alternating_polymer = 'alternating'

pmb.define_particle(name = PDha_backbone_bead, q=0,diameter=0.4*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle(name = PDha_carboxyl_bead, q=0, diameter=0.5*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle(name = PDha_amine_bead, q=0,diameter=0.3*pmb.units.nm,epsilon=1*pmb.units('reduced_energy'))

pmb.define_particle( name = PDAGA_backbone_bead, q=0,diameter=0.4*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_amine_bead,   q = 0, diameter=0.3*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_alpha_carboxyl_bead, q = 0, diameter=0.2*pmb.units.nm, epsilon=1*pmb.units('reduced_energy'))
pmb.define_particle( name = PDAGA_beta_carboxyl_bead, q = 0, diameter = 0.4*pmb.units.nm, epsilon = 1*pmb.units('reduced_energy'))


# Define a residue for the side chain 

PDAGA_side_chain_residue = 'PDAGA_side_chain_residue'
pmb.define_residue (name=PDAGA_side_chain_residue,
                    central_bead=PDAGA_amine_bead,
                    side_chains=[PDAGA_alpha_carboxyl_bead,PDAGA_beta_carboxyl_bead])

pmb.define_residue(name = PDha_residue, central_bead = PDha_backbone_bead ,
                    side_chains=[PDha_carboxyl_bead,PDha_amine_bead])

pmb.define_residue( name = PDAGA_monomer_residue,
                    central_bead = PDAGA_backbone_bead,
                    side_chains = [PDAGA_side_chain_residue])

pmb.define_molecule(name=alternating_polymer, residue_list=[PDha_residue,PDAGA_monomer_residue]*N_monomers)


#Define the bonds in the residues

pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDAGA_backbone_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_carboxyl_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_amine_bead)
pmb.define_bond(bond_object=generic_bond, particle_name1=PDha_backbone_bead, particle_name2=PDha_backbone_bead)

pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_backbone_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_backbone_bead, particle_name2=PDAGA_amine_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_alpha_carboxyl_bead, particle_name2=PDAGA_amine_bead)
pmb.define_bond(bond_object =generic_bond, particle_name1=PDAGA_beta_carboxyl_bead, particle_name2=PDAGA_amine_bead)

pmb.add_bonds_to_espresso(espresso_system=espresso_system)

# Create the alternating polymer into the espresso system

N_pol=1
pmb.create_pmb_object_in_espresso(name=alternating_polymer,
                                  number_of_objects=N_pol,
                                  espresso_system=espresso_system,
                                  position=[[Box_L.to('reduced_length').magnitude/2]*3]) #Create it in the box center 

picture_name='alternating_system.png'

do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=alternating_polymer, espresso_system=espresso_system)

### Use pyMBE to create peptides <a class="anchor" id="peptides"></a>

pyMBE includes built-on functions to facilitate the setting up of coarse-grained models for peptides from their aminoacid sequence. At the moment, two different coarse-grained models have been implemented with one bead (`model=1beadAA`) or two beads (`model=2beadAA`) per aminoacid. For the aminoacids currently available, we provide reference parameters in the folder (`reference_parameters`) which can be loaded into pyMBE

In [16]:
N_peptide=1
sequence="KKKKKEEEEE"
model='2beadAA'  # Model with 2 beads per each aminoacid

 # Load peptide parametrization from Lunkad, R. et al.  Molecular Systems Design & Engineering (2021), 6(2), 122-131.

pmb.load_interaction_parameters (filename='reference_parameters/interaction_parameters/Lunkad2021.txt')
pmb.add_bonds_to_espresso (espresso_system=espresso_system) # Add the bonds loaded from the file to the espresso system

pmb.define_peptide(name=sequence, sequence=sequence, model=model)

pmb.create_pmb_object_in_espresso(name=sequence,
                                  number_of_objects=N_peptide,
                                  espresso_system=espresso_system,
                                  position=[[Box_L.to('reduced_length').magnitude/2]*3]) #Create it in the box center 

#pmb.setup_lj_interactions_in_espresso(espresso_system=espresso_system)
espresso_system.time_step=1e-3
espresso_system.thermostat.set_langevin(kT=1, gamma=1.0, seed=24)
espresso_system.cell_system.tune_skin(min_skin=0.01, max_skin =4.0, tol=0.1, int_steps=1000)

picture_name='peptide_system.png'
do_snapshot_espresso_system(espresso_system=espresso_system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
pmb.destroy_pmb_object_in_system(name=sequence, espresso_system=espresso_system)

Note that in `sequence` we have used the one letter code for the aminoacids. Internally, pyMBE uses the one-letter code to define each aminoacid. The three letter code can also be used as input, which Sugar automatically detects and transforms into the one-letter code using its own protein sequence parser.

In [17]:
# The same peptide as before, but now using the three letter code for aminoacids

sequence='LYS-LYS-LYS-LYS-LYS-GLU-GLU-GLU-GLU-GLU'
pmb.define_peptide(name=sequence, sequence=sequence, model=model)
print('one letter code', pmb.protein_sequence_parser(sequence=sequence))
print('defined peptide sequence ', sequence)


one letter code ['K', 'K', 'K', 'K', 'K', 'E', 'E', 'E', 'E', 'E']
defined peptide sequence  LYS-LYS-LYS-LYS-LYS-GLU-GLU-GLU-GLU-GLU
